# Model Explore

In [ ]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.5 MB/s eta 0:00:00


In [ ]:
pip install nltk rouge bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
import json

In [ ]:
notebook_login()

# more model

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load model with 8-bit precision (needs less memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # For lower RAM usage (needs `bitsandbytes`)
    torch_dtype=torch.float16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# random parameter

In [ ]:
def answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " answer this question : when  can I use panadol?"
answer(prompt)

 answer this question : when  can I use panadol?
A. Panadol is used to reduce the fever, relieve the pain and inflammation in the body.
B. Panadol is used to reduce the fever, relieve the pain and inflammation in the


' answer this question : when  can I use panadol?\nA. Panadol is used to reduce the fever, relieve the pain and inflammation in the body.\nB. Panadol is used to reduce the fever, relieve the pain and inflammation in the'

In [ ]:
prompt = "I have headache which medicine should i take?"
answer(prompt)

I have headache which medicine should i take?
Is it safe to take 1200 mg of ibuprofen a day?
Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by reducing substances


'I have headache which medicine should i take?\nIs it safe to take 1200 mg of ibuprofen a day?\nIbuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by reducing substances'

# Question asking

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the benchmarking questions
with open('/content/benchmarking.json', 'r') as file:
    benchmarking_data = json.load(file)

In [ ]:


results = []

# Iterate over each question, use the model to generate an answer, and save the results
for item in benchmarking_data:
    question = item['question']
    generated_answer = answer(question)
    results.append({
        'question': question,
        'expected_answer': item['answer'],
        'generated_answer': generated_answer
    })






What are common causes of headaches?
There are many causes of headaches, but some of the more common ones include:
Cluster headaches: These headaches occur in groups and typically affect one side of the head. They are often associated with other symptoms, such as
How can you differentiate between a tension headache and a migraine?
How can you differentiate between a tension headache and a migraine? What are the causes of tension headaches? What are the causes of migraines?
A tension headache is a mild to moderate pain
What home remedies are effective for relieving headaches?
What are the best home remedies for headaches?
What foods can cause headaches?
What is the best home remedy for headache?
How do I stop getting headaches?
What is the best home rem
What are typical causes of lower back pain?
Dr. Mark Schroeder answered
Pain: The most common cause of back pain is muscle strain, often from lifting or twisting incorrectly. This is often accompanied by muscle spasm and can be treated w

In [ ]:
# Save the results to a new JSON file for later analysis
with open('model_before_answers.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)

print("Answer generation complete. Results saved to 'model_answers.json'.")

Answer generation complete. Results saved to 'model_answers.json'.


# Evaluation before hyperparamter tuning

## reuse

In [ ]:
pip install nltk rouge bert-score


In [ ]:
pip install sacrebleu

In [ ]:
import nltk
nltk.download('punkt')  # Download necessary data for word tokenization
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('wordnet')
from nltk.translate.meteor_score import meteor_score

nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score
import sacrebleu

In [ ]:
import scipy.spatial
from sentence_transformers import SentenceTransformer
# Load the sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')


# Function to calculate cosine similarity
def calculate_similarity(reference, candidate):
    emb1 = model.encode(reference)
    emb2 = model.encode(candidate)
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

## no reuse

In [ ]:
# Load the data
with open('/content/drive/MyDrive/llm/model_before_answers.json', 'r') as file:
    data = json.load(file)

references = [item['expected_answer'] for item in data]
candidates = [item['generated_answer'] for item in data]

## Blue

In [ ]:
# Prepare data for BLEU score
bleu_references = [[ref.split()] for ref in references]  # BLEU expects a list of list of tokens
bleu_candidates = [cand.split() for cand in candidates]

# Calculate BLEU score
bleu_score = corpus_bleu(bleu_references, bleu_candidates)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 0.018365129100319


## Rouge

In [ ]:
rouge = Rouge()
rouge_scores = rouge.get_scores(candidates, references, avg=True)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge-1': {'r': 0.2083327370136514, 'p': 0.13953207019063926, 'f': 0.16222520573015256}, 'rouge-2': {'r': 0.06774041799676293, 'p': 0.03536968433700631, 'f': 0.045267280609500686}, 'rouge-l': {'r': 0.19016640263935092, 'p': 0.12651104179122089, 'f': 0.1474447105381115}}


## meteor

In [ ]:
meteor_scores = [meteor_score([word_tokenize(ref)], word_tokenize(cand)) for ref, cand in zip(references, candidates)]
average_meteor = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR Score: {average_meteor}")

Average METEOR Score: 0.1879472613119189


## BertSCORE

In [ ]:
P, R, F1 = bert_score(candidates, references, lang="en", verbose=True)
average_bertscore = F1.mean().item()
print(f"Average BERTScore: {average_bertscore}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 2.71 seconds, 61.33 sentences/sec
Average BERTScore: 0.8463773727416992


## chrf_scores

In [ ]:
chrf_scores = [sacrebleu.metrics.CHRF().corpus_score([cand], [[ref]]) for cand, ref in zip(candidates, references)]
average_chrf = sum(score.score for score in chrf_scores) / len(chrf_scores)
print(f"Average chrF Score: {average_chrf}")


Average chrF Score: 29.414598408426983


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:



# Calculate semantic similarities
similarities = []
for item in data:
    expected_answer = item['expected_answer']
    generated_answer = item['generated_answer']
    similarity = calculate_similarity(expected_answer, generated_answer)
    similarities.append(similarity)




In [ ]:
# Calculate the average semantic similarity
average_similarity = sum(similarities) / len(similarities)
print(f"Average Semantic Similarity: {average_similarity:.4f}")

Average Semantic Similarity: 0.5636


# All score before hyperparamter tuning

In [ ]:
# Print all metrics at once
print(f"BLEU Score: {bleu_score} \n")
print("ROUGE Scores:", rouge_scores)
print(f"\nAverage METEOR Score: {average_meteor}\n")
print(f"Average BERTScore score: {average_bertscore}\n")
print(f"Average chrF Score: {average_chrf}\n")
print(f"Average Semantic Similarity: {average_similarity:.4f}")

BLEU Score: 0.018365129100319 

ROUGE Scores: {'rouge-1': {'r': 0.2083327370136514, 'p': 0.13953207019063926, 'f': 0.16222520573015256}, 'rouge-2': {'r': 0.06774041799676293, 'p': 0.03536968433700631, 'f': 0.045267280609500686}, 'rouge-l': {'r': 0.19016640263935092, 'p': 0.12651104179122089, 'f': 0.1474447105381115}}

Average METEOR Score: 0.1879472613119189

Average BERTScore score: 0.8463773727416992

Average chrF Score: 29.414598408426983

Average Semantic Similarity: 0.5636


# Hyperparamter tuning

In [ ]:
import json
import itertools
from bert_score import score as bert_score
import random

In [ ]:
# Load the data
with open('/content/model_before_answers.json', 'r') as file:
    full_data = json.load(file)


# references = [item['expected_answer'] for item in data]
# candidates = [item['generated_answer'] for item in data]

In [ ]:
len(full_data)

166

In [ ]:
data = random.sample(full_data, 50)


In [ ]:
len(data)

50

In [ ]:
# Function to evaluate using BERTScore
def evaluate_output(generated_text, expected_text):
    P, R, F1 = bert_score([generated_text], [expected_text], lang="en", verbose=False)
    return F1.mean().item()  # Using the F1 score as the metric
# Define hyperparameters grid



In [ ]:
# Define the range for temperature
temperature_values = [0.5, 0.7, 0.9]
default_top_p = 0.9
default_top_k = 40

results_temperature = {}
for temperature in temperature_values:
    scores = []
    for item in data:
        prompt = item['question']
        expected_answer = item['expected_answer']
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_length=50,
            do_sample=True,
            temperature=temperature,
            top_p=default_top_p,
            top_k=default_top_k
        )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        score = evaluate_output(generated_text, expected_answer)
        scores.append(score)
    average_score = sum(scores) / len(scores)
    results_temperature[temperature] = average_score




Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  27%|##7       | 388M/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

In [ ]:
best_temperature = max(results_temperature, key=results_temperature.get)
print(f"Best temperature: {best_temperature} with score: {results_temperature[best_temperature]}")

Best temperature: 0.5 with score: 0.8524714255332947


In [ ]:
top_p_values = [0.8, 0.9, 1.0]
best_temperature = 0.5  # From previous results

results_top_p = {}
for top_p in top_p_values:
    scores = []
    for item in data:
        prompt = item['question']
        expected_answer = item['expected_answer']
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_length=50,
            do_sample=True,
            temperature=best_temperature,
            top_p=top_p,
            top_k=default_top_k
        )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        score = evaluate_output(generated_text, expected_answer)
        scores.append(score)
    average_score = sum(scores) / len(scores)
    results_top_p[top_p] = average_score




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

In [ ]:
best_top_p = max(results_top_p, key=results_top_p.get)
print(f"Best top_p: {best_top_p} with score: {results_top_p[best_top_p]}")

Best top_p: 1.0 with score: 0.8493473565578461


In [ ]:
top_k_values = [20, 40, 60]
best_temperature = 0.5  # From previous results
best_top_p = 1       # From previous results

results_top_k = {}
for top_k in top_k_values:
    scores = []
    for item in data:
        prompt = item['question']
        expected_answer = item['expected_answer']
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_length=50,
            do_sample=True,
            temperature=best_temperature,
            top_p=best_top_p,
            top_k=top_k
        )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        score = evaluate_output(generated_text, expected_answer)
        scores.append(score)
    average_score = sum(scores) / len(scores)
    results_top_k[top_k] = average_score




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

In [ ]:
best_top_k = max(results_top_k, key=results_top_k.get)
print(f"Best top_k: {best_top_k} with score: {results_top_k[best_top_k]}")

Best top_k: 60 with score: 0.8518001580238342


In [ ]:
print(f"Best temperature: {best_temperature} with score: {results_temperature[best_temperature]}")
print(f"Best top_p: {best_top_p} with score: {results_top_p[best_top_p]}")
print(f"Best top_k: {best_top_k} with score: {results_top_k[best_top_k]}")


Best temperature: 0.5 with score: 0.8524714255332947
Best top_p: 1 with score: 0.8493473565578461
Best top_k: 60 with score: 0.8518001580238342


# after hyperparamter tuning

In [ ]:
def answer_after_fine(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.5,
        top_p=1,
        top_k=60,
        eos_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " answer this question : when  can I use panadol?"
answer_after_fine(prompt)

 answer this question : when  can I use panadol?
Panadol is a brand name for paracetamol. It is used to treat mild to moderate pain (e.g. headache, toothache, backache, sore throat) and fever.


' answer this question : when  can I use panadol?\nPanadol is a brand name for paracetamol. It is used to treat mild to moderate pain (e.g. headache, toothache, backache, sore throat) and fever.'

In [ ]:
results_after_fine = []

# Iterate over each question, use the model to generate an answer, and save the results
for item in benchmarking_data:
    question = item['question']
    generated_answer = answer_after_fine(question)
    results_after_fine.append({
        'question': question,
        'expected_answer': item['answer'],
        'generated_answer': generated_answer
    })

What are common causes of headaches?
What are the most common causes of headaches?
What are the most common causes of headaches? The most common causes of headaches are tension-type headaches, migraine headaches, and cluster headaches. Tension-
How can you differentiate between a tension headache and a migraine?
Tension headaches are often described as a pressing or tightening sensation. Migraines, on the other hand, are often accompanied by nausea, vomiting, and sensitivity to light and sound.
What is the best
What home remedies are effective for relieving headaches?
What is the best home remedy for a headache?
What is the best home remedy for headache?
What is the best home remedy for headache pain?
What is the best medicine for headache?
What
What are typical causes of lower back pain?
What are typical causes of lower back pain? Lower back pain is a common problem, and it can be caused by a variety of factors. Some of the most common causes of lower back pain include:
Spinal injurie

In [ ]:
# Save the results to a new JSON file for later analysis
with open('model_answer_after_hyper.json', 'w') as outfile:
    json.dump(results_after_fine, outfile, indent=4)

print("Answer generation complete. Results saved to 'model_answer_after_hyper.json'.")

Answer generation complete. Results saved to 'model_answer_after_hyper.json'.


In [ ]:
# Load the data after hyperparameter tuning
with open('/content/model_answer_after_hyper.json', 'r') as file:
    data_after = json.load(file)

refs_after = [item['expected_answer'] for item in data_after]
cands_after = [item['generated_answer'] for item in data_after]


In [ ]:
## BLEU
bleu_refs_after = [[ref.split()] for ref in refs_after]
bleu_cands_after = [cand.split() for cand in cands_after]
bleu_score_after = corpus_bleu(bleu_refs_after, bleu_cands_after)

In [ ]:
print(f"BLEU Score After: {bleu_score_after}\n")

BLEU Score After: 0.021898307634920827



In [ ]:


## ROUGE
rouge_after = Rouge()
rouge_scores_after = rouge_after.get_scores(cands_after, refs_after, avg=True)

## METEOR
meteor_scores_after = [meteor_score([word_tokenize(ref)], word_tokenize(cand)) for ref, cand in zip(refs_after, cands_after)]
average_meteor_after = sum(meteor_scores_after) / len(meteor_scores_after)

## BERTScore
P_after, R_after, F1_after = bert_score(cands_after, refs_after, lang="en", verbose=True)
average_bertscore_after = F1_after.mean().item()

## chrF
chrf_scores_after = [sacrebleu.metrics.CHRF().corpus_score([cand], [[ref]]) for cand, ref in zip(cands_after, refs_after)]
average_chrf_after = sum(score.score for score in chrf_scores_after) / len(chrf_scores_after)




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 2.70 seconds, 61.37 sentences/sec


In [ ]:
## Semantic Similarity
similarities_after = []
for item in data_after:
    expected_answer = item['expected_answer']
    generated_answer = item['generated_answer']
    similarity = calculate_similarity(expected_answer, generated_answer)
    similarities_after.append(similarity)
average_similarity_after = sum(similarities_after) / len(similarities_after)

In [ ]:
print(f"BLEU Score After: {bleu_score_after}\n")
print("ROUGE Scores After:", rouge_scores_after)
print(f"\nAverage METEOR Score After: {average_meteor_after}\n")
print(f"Average BERTScore After: {average_bertscore_after}\n")
print(f"Average chrF Score After: {average_chrf_after}\n")
print(f"Average Semantic Similarity After: {average_similarity_after:.4f}")


BLEU Score After: 0.021898307634920827

ROUGE Scores After: {'rouge-1': {'r': 0.20629963994949482, 'p': 0.14922652234802114, 'f': 0.1678071804988965}, 'rouge-2': {'r': 0.07138029126709163, 'p': 0.04057446320871533, 'f': 0.05015519389739996}, 'rouge-l': {'r': 0.1904219558088973, 'p': 0.13757772836737991, 'f': 0.15455426517499718}}

Average METEOR Score After: 0.18235965287193587

Average BERTScore After: 0.8460686802864075

Average chrF Score After: 29.417746841236447

Average Semantic Similarity After: 0.5466


| Metric                      | Before Tuning        | After Tuning         |
|-----------------------------|----------------------|----------------------|
| BLEU Score                  | 0.018365129100319    | 0.021898307634920827 |
| ROUGE-1 Recall              | 0.2083327370136514   | 0.20629963994949482  |
| ROUGE-1 Precision           | 0.13953207019063926  | 0.14922652234802114  |
| ROUGE-1 F1-Score            | 0.16222520573015256  | 0.1678071804988965   |
| ROUGE-2 Recall              | 0.06774041799676293  | 0.07138029126709163  |
| ROUGE-2 Precision           | 0.03536968433700631  | 0.04057446320871533  |
| ROUGE-2 F1-Score            | 0.045267280609500686 | 0.05015519389739996  |
| ROUGE-L Recall              | 0.19016640263935092  | 0.1904219558088973   |
| ROUGE-L Precision           | 0.12651104179122089  | 0.13757772836737991  |
| ROUGE-L F1-Score            | 0.1474447105381115   | 0.15455426517499718  |
| Average METEOR Score        | 0.1879472613119189   | 0.18235965287193587  |
| Average BERTScore           | 0.8463773727416992   | 0.8460686802864075   |
| Average chrF Score          | 29.414598408426983   | 29.417746841236447   |
| Average Semantic Similarity | 0.5636               | 0.5466               |


BLEU and ROUGE scores improved slightly → better lexical match

METEOR and Semantic Similarity decreased → slightly weaker meaning alignment

BERTScore and chrF remained almost the same → no major semantic or character-level change